In [2]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
model = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [9]:
ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can you explain Dynamic Programming under 50 words?")
        ],
    }
)

print(ai_msg.content)

Dynamic Programming is a method for solving complex problems by breaking them down into simpler subproblems, solving each subproblem just once, and storing their solutions. It is often used for optimization problems where overlapping subproblems and optimal substructure properties exist.


In [12]:
## Method 1: Pass Chat History to your LLM as a context.

chat_history = [
    HumanMessage(content="Can you explain Dynamic Programming under 50 words?"),
    AIMessage(content=ai_msg.content)
]

chat_history

[HumanMessage(content='Can you explain Dynamic Programming under 50 words?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Dynamic Programming is a method for solving complex problems by breaking them down into simpler subproblems, solving each subproblem just once, and storing their solutions. It is often used for optimization problems where overlapping subproblems and optimal substructure properties exist.', additional_kwargs={}, response_metadata={})]

In [19]:
def interrogate_model(chat_history: list, preserve_chat_history=True):
    current_message = [HumanMessage(content="What did I ask you earlier?")]
    chat_messages = []

    if preserve_chat_history:
        chat_messages = chat_history + current_message
    else:
        chat_messages = current_message


    ai_msg = chain.invoke(
        {
            "messages": chat_messages
        }
    )

    return ai_msg.content

In [21]:
interrogate_model(chat_history, preserve_chat_history=False)

"I'm sorry, but I don't have the ability to recall past conversations or remember any previous interactions. Each session is treated independently. How can I assist you today?"

In [39]:
# Method 2: Summarize the chat history

def summarize_chat_history(chat_history: list):
    """ Summarizes the chat history when the message count is greater than 4."""

    if len(chat_history) >= 4:
        summary_prompt = (
            "Distill the above chat messages into a single summary message. "
            "Include as many specific details as you can."
        )
        print(chat_history)
        summary_message = chain.invoke(
            chat_history + [HumanMessage(content=summary_prompt)]
        )

        return summary_message.content
    
    return ""


In [43]:
def interrogate_model_with_summary(query: str, chat_history: list, debug_mode=False):
    current_message = [HumanMessage(content=query)]
    chat_messages = []

    if len(chat_history) >= 4:
        print("Summarizing your content...")
        summary_message = summarize_chat_history(chat_history)
        chat_messages = [AIMessage(content=summary_message)] + current_message
    else:
        chat_messages = chat_history + current_message
    
    if debug_mode:
        print("Messages sent to the model:")
        print(chat_messages, "\n")

    ai_msg = chain.invoke(
        {
            "messages": chat_messages
        }
    )

    return AIMessage(content=ai_msg.content)

In [33]:
def chat_with_model(query: str, chat_history: list, debug_mode=False):
    query_response = interrogate_model_with_summary(query, chat_history, debug_mode)
    
    # Append the query and response to the chat history
    chat_history.append(HumanMessage(content=query))
    chat_history.append(query_response)
    
    return query_response, chat_history

In [54]:
chat_history = []

query1 = "Can you explain Dynamic Programming under 50 words?"
query_response1, chat_history = chat_with_model(query1, chat_history, debug_mode=True)

print(query_response1.content)

Messages sent to the model:
[HumanMessage(content='Can you explain Dynamic Programming under 50 words?', additional_kwargs={}, response_metadata={})] 

Dynamic Programming is a computational method used to solve complex problems by breaking them down into simpler subproblems, storing the results of these subproblems to avoid redundant calculations. It is commonly used in optimization and decision-making tasks, such as the Fibonacci sequence and shortest path algorithms.


In [55]:
query2 = "Can you explain Graph data structures under 50 words?"
query_response2, chat_history = chat_with_model(query2, chat_history, debug_mode=True)

print(query_response2.content)

Messages sent to the model:
[HumanMessage(content='Can you explain Dynamic Programming under 50 words?', additional_kwargs={}, response_metadata={}), AIMessage(content='Dynamic Programming is a computational method used to solve complex problems by breaking them down into simpler subproblems, storing the results of these subproblems to avoid redundant calculations. It is commonly used in optimization and decision-making tasks, such as the Fibonacci sequence and shortest path algorithms.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you explain Graph data structures under 50 words?', additional_kwargs={}, response_metadata={})] 

Graph data structures consist of vertices (nodes) and edges (connections) representing relationships between entities. They can be directed or undirected, weighted or unweighted, and are used to model networks, such as social connections, transportation systems, and computer networks, enabling efficient traversal and manipulation.


In [56]:
query3 = "Can you tell me what did I ask till now?"
query_response3, chat_history = chat_with_model(query3, chat_history, debug_mode=True)

print(query_response3.content)

Summarizing your content...
[HumanMessage(content='Can you explain Dynamic Programming under 50 words?', additional_kwargs={}, response_metadata={}), AIMessage(content='Dynamic Programming is a computational method used to solve complex problems by breaking them down into simpler subproblems, storing the results of these subproblems to avoid redundant calculations. It is commonly used in optimization and decision-making tasks, such as the Fibonacci sequence and shortest path algorithms.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you explain Graph data structures under 50 words?', additional_kwargs={}, response_metadata={}), AIMessage(content='Graph data structures consist of vertices (nodes) and edges (connections) representing relationships between entities. They can be directed or undirected, weighted or unweighted, and are used to model networks, such as social connections, transportation systems, and computer networks, enabling efficient traversal and 